# Assignent 3
### Team 26

## Importing the dataset

In [133]:
import pandas as pd
import itertools

In [134]:
df = pd.read_csv("../myDataFile.csv", low_memory=False)
df.fillna(0, inplace=True)
df.replace('t', 1, inplace=True)
df.to_csv("../data.csv")

In [135]:
df

,Instant_food_products,UHT_milk,abrasive_cleaner,artif__sweetener,baby_cosmetics,baby_food,bags,baking_powder,bathroom_cleaner,beef,...,turkey,vinegar,waffles,whipped_sour_cream,whisky,white_bread,white_wine,whole_milk,yogurt,zwieback
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9830,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,1,0,0
9831,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9832,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9833,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Computing the frequent item sets
The code below computes the frequent item sets per layer.

### Apriori algorithm
We first generate the candidates for a given layer using self-joining. For each item want to add to the candidate list however, we first check if it holds up to pruning. Only then do we add it.

#### Self-joining
This step is performed inside the function below. We take each pair of frequent item sets from the previous layer and create a potential candidate for the current layer by merging the 2 item sets. We make sure the two item sets of the previous layer only differ by one element.

#### Pruning
Before appending a potential candidate to the candidates collection, we check if all its k-1 subsets are frequent.

In [136]:
def compute_frequent_item_sets(df, min_support):

    # Define first layer candidates in a dictionary
    candidates = {
        1: [{item} for item in list(df.columns)]
    }

    # Compute first layer frequent item sets.
    layers = {
        1: pick_candidates_for_layer(candidates[1], df, min_support)
    }

    n_columns = df.shape[1]
    for layer in range(2, n_columns + 1):

        # Step 1 - Self-join. Compute candidates for current layer
        candidates[layer] = list()
        for p in range(len(layers[layer - 1]) - 1):

            for q in range(p + 1, len(layers[layer - 1])):  # start at p+1 to avoid duplicates.

                p_set = layers[layer - 1][p]
                q_set = layers[layer - 1][q]

                # Check if the number of common elements between the two item sets equals layer - 2.
                # If so, they differ by one element and a new candidate (union) can be computed.
                if len(p_set.intersection(q_set)) == layer - 2:
                    new_candidate = p_set.union(q_set)

                    # Only append if not a duplicate
                    if not new_candidate in candidates[layer]:

                        # Step 2 - Pruning (based on existence of subsets in previous layer)
                        subsets_of_new_candidate = set(itertools.combinations(new_candidate, layer - 1))
                        if all([set(subset) in layers[layer - 1] for subset in subsets_of_new_candidate]):
                            candidates[layer].append(new_candidate)


        # Pick item sets for layer if they exceed the minimum support.
        picked = pick_candidates_for_layer(candidates[layer], df, min_support)

        # Break early if no more frequent item sets are found.
        if len(picked) == 0:
            break

        layers[layer] = picked

        print("Finished computing for layer " + str(layer))

    return layers, candidates

### Computing support
The function below returns the support metric for a given `item_set`. We first need to check if the `item_set` is valid (i.e. has at least one item) before computing. This computation is done by simply looking up values using a Pandas DataFrame query.

In [137]:
def compute_support_for(df, item_set):
    # df is the (reference to) complete dataframe
    # item_set is a set of strings, each being a column name in df to be checked.
    # returns the support percentage for a given item_set.

    if len(item_set) == 0:
        return 0

    query = " == 1 and ".join(item_set) + " == 1"
    return df.query(query).shape[0] / df.shape[0]

### Picking candidates
The following function prunes the candidate itemsets after evaluating their support. If they exceed the minimum value passed as a parameter, then the itemset will go into L (the final frequent itemsets collection)

In [138]:
def pick_candidates_for_layer(candidates, df, minSup):
    return [item_set for item_set in candidates if compute_support_for(df, item_set) >= minSup]

## Rule generation
The method below generates all rules for a given collection of frequent item sets. It does this by, for each layer and each frequent item set in that layer, computing the available rules for that item set.

Note that we skip the first layer, as no rules can be generated using only one item in the item set.

In [139]:
def generate_all_rules(df, layers, min_conf, rules):

    # For each layer of the item sets
    for k, item_sets in layers.items():

        # No rules can be formed out of item sets with length 1.
        if k == 1:
            continue

        # Per item set in this layer...
        for item_set in item_sets:

            # ... compute the rules
            compute_rules_for(df, item_set, set([]), min_conf, rules)

        print("Computed rules for item sets of length " + str(k) + ":")

### Computing rules from an item set
Given an item set in the layer, we use a recursive method to generate all possible rules from that item set.

In order to do this we use the non-monotonicity property. This means we start out with a large antecedent, and as we go deeper into the recursion we move element from the antecedent set to the consequent set. We stop the branch of the recursion if the rule does not mean the minimum confidence. Therefor we use the non-monotonicity property to stop early with checking possble rules. Moreover, we skip possible duplicates by checking if the rule is already in the dictionary.

For each (unique) possible rule, we then compute the confidence level and store the rule if it meets or exceeds the threshold, as we've interpreted the minimum confidence this way.

Note that, due to Python's pass-by-reference policy, we need to copy existing item sets, in order to not lose original item set we branch from.

In [140]:
def compute_rules_for(df, antecedents, consequents, min_conf, rules):

    # Base case, no more consequents for valid rule.
    if len(antecedents) == 0:
        return rules

    # For each consequent
    for antecedent in antecedents:

        next_consequents = consequents.copy()
        next_consequents.add(antecedent)  # add single antecedent to consequent set.

        # Create copy (because python) of antecedents and remove one antecedent.
        next_antecedents = antecedents.copy()
        next_antecedents.remove(antecedent)

        # Skip duplicates
        if frozenset(next_antecedents) in rules and frozenset(next_consequents) in rules[frozenset(next_antecedents)]:
            continue

        # Compute confidence for possible new rule.
        conf = compute_confidence(df, next_antecedents, next_consequents)

        # If rule passes min confidence, add to rules
        if conf >= min_conf:

            # Add to the dict
            if frozenset(next_antecedents) in rules:
                rules[frozenset(next_antecedents)][frozenset(next_consequents)] = conf
            else:
                rules[frozenset(next_antecedents)] = {
                    frozenset(next_consequents) : conf
                }

            if len(next_antecedents) > 1:
                # Only then, go into recursion on these item sets.
                compute_rules_for(df, next_antecedents, next_consequents, min_conf, rules)


### Computing confidence
Lastly, computing the confidence level is done simply by using the previously mentioned `compute_support_for()` method in the formula for computing confidence.

In [141]:
# takes 3 input parameters, the reference to the dataframe and 2 item sets
def compute_confidence(df, antecedent, consequent):
    return compute_support_for(df, set.union(antecedent, consequent)) / compute_support_for(df, antecedent)

## Output

In [142]:
# First, compute item sets
L, C = compute_frequent_item_sets(df, 0.005)

Finished computing for layer 2
Finished computing for layer 3
Finished computing for layer 4


In [143]:
rules = {}

# Then generate the rules
generate_all_rules(df, L, 0.6, rules)

Computed rules for item sets of length 2:
Computed rules for item sets of length 3:
Computed rules for item sets of length 4:


In [144]:
# Show results for computing the item sets
print("Lengths for L:")
for key, value in L.items():
    print(str(key) + ": " + str(len(value)))

Lengths for L:
1: 120
2: 605
3: 264
4: 12


In [145]:
# Method to prettify frozenset strings.
def pretty(item_set):
    return "{" + ", ".join(item_set) + "}"

# Show results for the rules
count = 0
for antecedent in rules.keys():
    # print("Ant: " + str(antecedent) + ", con: " + str(rules[antecedent]))
    count += len(rules[antecedent].keys())

    for consequent in rules[antecedent].keys():
        print(pretty(antecedent) + " -> " + pretty(consequent) + " (confidence = " + str(rules[antecedent][consequent]) + ")")

print("Rule count: " + str(count))

{bottled_water, butter} -> {whole_milk} (confidence = 0.6022727272727273)
{domestic_eggs, butter} -> {whole_milk} (confidence = 0.6210526315789474)
{butter, root_vegetables} -> {whole_milk} (confidence = 0.6377952755905512)
{tropical_fruit, butter} -> {whole_milk} (confidence = 0.6224489795918368)
{whipped_sour_cream, butter} -> {whole_milk} (confidence = 0.66)
{butter, yogurt} -> {whole_milk} (confidence = 0.6388888888888888)
{tropical_fruit, curd} -> {whole_milk} (confidence = 0.6336633663366337)
{domestic_eggs, margarine} -> {whole_milk} (confidence = 0.6219512195121952)
{pip_fruit, domestic_eggs} -> {whole_milk} (confidence = 0.6235294117647059)
{domestic_eggs, tropical_fruit} -> {whole_milk} (confidence = 0.6071428571428571)
{onions, root_vegetables} -> {other_vegetables} (confidence = 0.6021505376344086)
{pip_fruit, whipped_sour_cream} -> {other_vegetables} (confidence = 0.6043956043956045)
{pip_fruit, whipped_sour_cream} -> {whole_milk} (confidence = 0.6483516483516485)
{whole_m